In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('./SD-Datasets')

Mounted at ./SD-Datasets


In [ ]:
train_dir = './SD-Datasets/MyDrive/SD-Datasets/Skin Disease_dataset/TRAIN' #ini gdrive pribadi resti
test_dir = './SD-Datasets/MyDrive/SD-Datasets/Skin Disease_dataset/TEST'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 800 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
22/22 [==============================] - 20s 777ms/step - loss: 1.1701 - accuracy: 0.5235 - val_loss: 1.5752 - val_accuracy: 0.4375
Epoch 2/10
22/22 [==============================] - 17s 772ms/step - loss: 1.0885 - accuracy: 0.5647 - val_loss: 1.5420 - val_accuracy: 0.4500
Epoch 3/10
22/22 [==============================] - 17s 772ms/step - loss: 1.0590 - accuracy: 0.5868 - val_loss: 1.5883 - val_accuracy: 0.4062
Epoch 4/10
22/22 [==============================] - 16s 743ms/step - loss: 1.0186 - accuracy: 0.6044 - val_loss: 1.6648 - val_accuracy: 0.3875
Epoch 5/10
22/22 [==============================] - 17s 775ms/step - loss: 0.9885 - accuracy: 0.6221 - val_loss: 1.7004 - val_accuracy: 0.4000
Epoch 6/10
22/22 [==============================] - 17s 775ms/step - loss: 0.9367 - accuracy: 0.6485 - val_loss: 1.6293 - val_accuracy: 0.4563
Epoch 7/10
22/22 [==============================] - 17s 763ms/step - loss: 0.9463 - accuracy: 0.6103 - val_loss: 1.5656 - val_accuracy: 0.4187